# Pipeline: O secuencia de funciones de preprocesamiento y entrenamiento

In [4]:
import numpy as np
import pandas as pd
# Entorno Scikit Learn
from sklearn.neighbors import KNeighborsRegressor   # Algoritmo
from sklearn.metrics import mean_squared_error, mean_absolute_error      # Metrica de desempeño ( para ver que tan bien se ajusta nuestro modelo a los datos)
from sklearn.model_selection import train_test_split    # Para dividir el conjunto de prueba y entrenamiento
from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score   # Validación cruzada 
from sklearn.model_selection import GridSearchCV
# Para gráficos
import matplotlib.pyplot as plt

In [5]:
# Datos
data=pd.read_csv("https://raw.githubusercontent.com/Albertuff/Machine-Learning/master/datos/estatura_peso.csv")
data.head()

,gender,height,weight
0,0,63.2,168.7
1,0,68.7,169.8
2,0,64.8,176.6
3,0,67.9,246.8
4,1,68.9,151.6


In [6]:
from sklearn.pipeline import Pipeline
from sklearn import preprocessing

X=data[["height"]]
Y=data["weight"]

# Primero se arreglan los datos faltantes si es que hay, luego se hace el reescalamiento y despues se agrega el modelo
Pipe=Pipeline([("zscore",preprocessing.StandardScaler()),("Modelo",KNeighborsRegressor())]) #Los preocesos se definen como una lista
# El hiperparámetro del modelo es: n_neighbors

# Definimos el método de busqueda
espacio_parametros={"Modelo__n_neighbors":np.arange(1,50)}  # Para pasar los hiperparámetros al pipe es necesario, anteponer al nomobre del hiperparámetro
                                                            # el nombre que le estamos dando en el pipe a nuestro algoritmo, enn este caso: Modelo, seguido de dos guiones bajos
                                                            # Modelo__

rejilla=GridSearchCV(Pipe,param_grid=espacio_parametros,scoring="neg_mean_absolute_error",n_jobs=-1,cv=10)
rejilla.fit(X,Y)
print(f" La mejor configuración del hiperparámetro k es: {rejilla.best_params_}")
print(f" El mejor score (error absoluto promedio) es: {-rejilla.best_score_}")

 La mejor configuración del hiperparámetro k es: {'Modelo__n_neighbors': 7}
 El mejor score (error absoluto promedio) es: 25.392857142857146


In [7]:
# Sabemos que este valor del score, es una metrica "optimista". Para tener uan metrica mas realista, hacemos una validacion cruzada anidada

# cv_interna=KFold(n_splits=10)
# cv_externa=KFold(n_splits=10)

rejilla=GridSearchCV(Pipe,param_grid=espacio_parametros,scoring="neg_mean_absolute_error",n_jobs=-1,cv=10)
puntajes=cross_val_score(rejilla,X,Y,cv=10,scoring="neg_mean_absolute_error",n_jobs=-1)
print(f" El mejor score (error absoluto promedio) es: {-puntajes.mean()}")

 El mejor score (error absoluto promedio) es: 27.241248150676103


In [8]:
# Otro tipo de validación cruzada

cv_interna=KFold(n_splits=10)
cv_externa=RepeatedKFold(n_splits=10,n_repeats=5)
rejilla=GridSearchCV(Pipe,param_grid=espacio_parametros,scoring="neg_mean_absolute_error",n_jobs=-1,cv=cv_interna)
puntajes=cross_val_score(rejilla,X,Y,cv=cv_externa,scoring="neg_mean_absolute_error",n_jobs=-1)
print(f" El mejor score (error absoluto promedio) es: {-puntajes.mean()}")

 El mejor score (error absoluto promedio) es: 27.72950544671992
